## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [4]:
# Let's just check emails are working for you
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jacobhuang91@gmail.com")  # Change to your verified sender
    to_email = To("jacobhuang91@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

UnauthorizedError: HTTP Error 401: Unauthorized

### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [5]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [6]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [7]:
# Runner.run_streamed 会启动一个 异步生成任务
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
# stream_events() 是一个异步生成器（async generator），会产生 事件对象（event）
async for event in result.stream_events():
    # event.type == "raw_response_event"：表示这是 Agent 的原始文本输出事件
    # ResponseTextDeltaEvent：表示 增量文本变化，也就是这次事件里生成的新文本片段（delta）
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        # event.data.delta 就是这一小段新生成的文本
        # end="" 保证不会自动换行
        # flush=True 保证立即打印到屏幕，而不是缓冲
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. I’m reaching out to introduce you to ComplAI, our cutting-edge SaaS solution designed to streamline SOC 2 compliance and audit preparation through the power of artificial intelligence.

Navigating the intricacies of SOC 2 requirements can be daunting. ComplAI simplifies this process, helping you efficiently manage compliance tasks, automate documentation, and ensure continuous monitoring—all while saving you time and resources. 

Here are a few benefits our clients have experienced:

- **Automated Workflows:** Reduce manual efforts with guided workflows that keep you on track.
- **Real-time Insights:** Gain clarity on compliance status and readiness for audits at any moment.
- **Expert Support:** Our team is dedicated to your success, offering insights and assistance along the way.

I would love to schedule a brief call to discuss how ComplAI can specifically benefit [Recip

In [8]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

# final_output 是 RunResult 对象的一个属性，用来表示 Agent 执行完后的完整文本输出
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out on behalf of ComplAI, a leading provider of AI-powered tools designed to simplify and streamline SOC 2 compliance.

Navigating the complexities of SOC 2 requirements can be challenging and time-consuming. Our platform offers an intuitive solution that automates compliance processes, reduces manual effort, and enhances audit preparedness. With ComplAI, you can:

- **Accelerate Compliance**: Our AI technology simplifies tracking and reporting, helping you meet regulatory standards efficiently.
- **Enhance Security**: Continuous monitoring and risk assessment capabilities ensure that your security practices stay up to date.
- **Prepare for Audits with Confidence**: Our tool prepares you thoroughly for audits, minimizing stress and ensuring a seamless experience.

I would love to schedule a brief call to discuss how ComplA

In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [ ]:
sales_agent1

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_email

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

你看到的这段话可以拆开解释：

### 1. **Tools（Agent-as-tools）**

* 一个 Agent 调用另一个 Agent 当作工具。
* 调用时：**控制权还会回到原来的 Agent**。
* 就像函数调用：A → 调用 B → B 返回结果 → A 继续执行。

例子：

* Email Manager 调用一个 `Grammar Agent` 来润色正文。
* Grammar Agent 返回润色好的文本。
* Email Manager 再继续执行后面的转换和发送。

### 2. **Handoffs**

* 一个 Agent 把任务整个交给另一个 Agent，**控制权彻底移交**。
* 原来的 Agent 不会等结果再回来继续，而是交出去后自己就退出了。
* 就像责任转移：A → 发现任务适合 B → 把控制权交给 B → 从此 B 负责到底。

例子：

* Orchestrator Agent 收到“发一封邮件”。
* 它判断这属于 Email Manager 的专业范围。
* 它执行 handoff，把整个任务交给 Email Manager，自己不再管。
* Email Manager 接管后，用自己的工具链完成“写标题 → 转 HTML → 发送”。

### 3. **那句话里的核心对比**

* **With tools, control passes back**：调用完工具，控制权回到调用方。
* **With handoffs, control passes across**：交给另一个 Agent，控制权完全转移过去。

所以在你 `handoff_description="Convert an email to HTML and send it"` 的例子里，这段描述就是 **告诉 Orchestrator 或别的 Agent：如果任务和这个描述匹配，你可以把控制权整个交给我，不用再接回去**。

要不要我给你写一个伪代码对比：

* Tool 调用流程
* Handoff 转移流程
  这样你可以一眼看到控制权差别？




In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.